In [ ]:
import torch
import numpy as np
import torch.utils.data as data
import torch.nn as nn
import pandas as pd
import time

In [1]:
# Prepare data
df = pd.read_csv(
    "data.csv",
    delimiter=",",
)

NameError: name 'pd' is not defined

In [ ]:
df.head()

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16
1,2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0,8,32,40
2,3,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0,5,27,32
3,4,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0,3,10,13
4,5,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0,0,1,1


In [ ]:
df = df.drop(columns=["instant", "dteday", "yr", "mnth", "casual", "registered"])

In [ ]:
df.head()

,season,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,cnt
0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,16
1,1,1,0,6,0,1,0.22,0.2727,0.80,0.0,40
2,1,2,0,6,0,1,0.22,0.2727,0.80,0.0,32
3,1,3,0,6,0,1,0.24,0.2879,0.75,0.0,13
4,1,4,0,6,0,1,0.24,0.2879,0.75,0.0,1


In [ ]:
train = df.sample(frac=0.8, random_state=200)  # random state is a seed value
test = df.drop(train.index)

print(train.shape)
print(test.shape)

train.head()

(8709, 11)
(2177, 11)


,season,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,cnt
4396,4,5,0,6,0,1,0.42,0.4242,0.71,0.1642,7
6205,1,19,0,2,1,2,0.36,0.3485,0.50,0.1642,281
2558,2,12,0,2,1,1,0.64,0.6212,0.41,0.3582,169
7225,2,10,0,4,1,1,0.48,0.4697,0.63,0.0896,189
3305,3,15,0,0,0,3,0.72,0.6970,0.74,0.2985,326


In [ ]:
train_dataset = data.TensorDataset(
    torch.from_numpy(train.values[:, :-1]), torch.from_numpy(train.values[:, -1])
)
test_dataset = data.TensorDataset(
    torch.from_numpy(test.values[:, :-1]), torch.from_numpy(test.values[:, -1])
)

next(iter(train_dataset))

(tensor([4.0000, 5.0000, 0.0000, 6.0000, 0.0000, 1.0000, 0.4200, 0.4242, 0.7100,
         0.1642], dtype=torch.float64),
 tensor(7., dtype=torch.float64))

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

# GPU operations have a separate seed we also want to set
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)
    torch.cuda.manual_seed_all(42)

# Additionally, some operations on a GPU are implemented stochastic for efficiency
# We want to ensure that all operations are deterministic on GPU (if used) for reproducibility
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False


class SimpleClassifier(nn.Module):
    def __init__(self, layer_sizes):
        super().__init__()
        self.layers = nn.ModuleList()
        for i in range(len(layer_sizes) - 1):
            self.layers.append(nn.Linear(layer_sizes[i], layer_sizes[i + 1]))
            if i < len(layer_sizes) - 2:  # Ostatnia warstwa nie powinna mieć funkcji aktywacji
                self.layers.append(nn.Tanh())

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

/home/kamil/.local/lib/python3.12/site-packages/torch/cuda/__init__.py:129: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at /pytorch/c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


In [ ]:
# Prepare model
device = torch.device("cuda")
model = SimpleClassifier([10, 200, 200, 1])
model.to(device)

RuntimeError: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero.

In [ ]:
import torch
import torch.nn as nn

class RMSLELoss(nn.Module):
    def __init__(self):
        super(RMSLELoss, self).__init__()

    def forward(self, y_pred, y_true):
        y_pred = torch.clamp(y_pred, min=0)  # Ensure non-negative predictions
        loss = torch.sqrt(torch.mean((torch.log1p(y_pred) - torch.log1p(y_true)) ** 2))
        return loss


In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_module = RMSLELoss()

train_data_loader = data.DataLoader(train_dataset, batch_size=100, shuffle=True)
test_data_loader = data.DataLoader(
    test_dataset, batch_size=len(test_dataset), shuffle=True, drop_last=False
)

In [ ]:
# Train model
model.train()

# Training loop
for epoch in range(600):
    for data_inputs, data_labels in train_data_loader:
        ## Step 1: Move input data to device (only strictly necessary if we use GPU)
        data_inputs = data_inputs.to(device)
        data_labels = data_labels.to(device)

        ## Step 2: Run the model on the input data
        preds = model(data_inputs.float())
        preds = preds.squeeze(
            dim=1
        )  # Output is [Batch size, 1], but we want [Batch size]

        ## Step 3: Calculate the loss
        loss = loss_module(preds, data_labels.float())

        ## Step 4: Perform backpropagation
        # Before calculating the gradients, we need to ensure that they are all zero.
        # The gradients would not be overwritten, but actually added to the existing ones.
        optimizer.zero_grad()
        # Perform backpropagation
        loss.backward()

        ## Step 5: Update the parameters
        optimizer.step()
    print(f"Epoch: {epoch+1}, loss: {loss.item():.3}")

Epoch: 1, loss: 2.58
Epoch: 2, loss: 1.75
Epoch: 3, loss: 2.28
Epoch: 4, loss: 1.46
Epoch: 5, loss: 1.31
Epoch: 6, loss: 1.22
Epoch: 7, loss: 1.24
Epoch: 8, loss: 1.45
Epoch: 9, loss: 0.917
Epoch: 10, loss: 0.805
Epoch: 11, loss: 0.982
Epoch: 12, loss: 1.26
Epoch: 13, loss: 1.13
Epoch: 14, loss: 0.75
Epoch: 15, loss: 0.506
Epoch: 16, loss: 0.755
Epoch: 17, loss: 1.02
Epoch: 18, loss: 1.19
Epoch: 19, loss: 1.06
Epoch: 20, loss: 0.492
Epoch: 21, loss: 0.661
Epoch: 22, loss: 0.483
Epoch: 23, loss: 0.649
Epoch: 24, loss: 0.96
Epoch: 25, loss: 0.887
Epoch: 26, loss: 0.694
Epoch: 27, loss: 0.63
Epoch: 28, loss: 0.74
Epoch: 29, loss: 0.972
Epoch: 30, loss: 0.601
Epoch: 31, loss: 0.535
Epoch: 32, loss: 0.902
Epoch: 33, loss: 0.695
Epoch: 34, loss: 0.48
Epoch: 35, loss: 0.706
Epoch: 36, loss: 0.564
Epoch: 37, loss: 0.407
Epoch: 38, loss: 0.512
Epoch: 39, loss: 0.445
Epoch: 40, loss: 0.679
Epoch: 41, loss: 0.718
Epoch: 42, loss: 0.468
Epoch: 43, loss: 0.564
Epoch: 44, loss: 0.512
Epoch: 45, loss

In [ ]:
def rmsle(y_true,y_pred):
    n = len(y_true)
    msle = np.mean([(np.log(max(y_pred[i],0) + 1) - np.log(y_true[i] + 1)) ** 2.0 for i in range(n)])
    return np.sqrt(msle)

In [ ]:
model.eval()  # Set model to eval mode
rmsle_errors = []

with torch.no_grad():  # Deactivate gradients for the following code
    for data_inputs, data_labels in test_data_loader:
        # Move data to the correct device
        data_inputs, data_labels = data_inputs.to(device), data_labels.to(device)

        # Model predictions
        preds = model(data_inputs.float()).squeeze(dim=1)

        # Convert predictions and labels to NumPy for RMSLE calculation
        preds_np = preds.cpu().numpy()
        labels_np = data_labels.cpu().numpy()

        # Compute RMSLE for this batch
        msle = rmsle(labels_np, preds_np)
        rmsle_errors.append(msle)

# Compute the mean RMSLE across all batches
mean_rmsle = np.mean(rmsle_errors)
print(f"Mean RMSLE Error: {mean_rmsle:.4f}")


Mean RMSLE Error: 0.4105


Model evaluation

In [ ]:
df_eval = pd.read_csv(
    "evaluation_data.csv",
    delimiter=",",
)
df_eval.head()

,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed
0,2011-01-20,1,0,1,0,0,4,1,1,0.26,0.2273,0.56,0.3881
1,2011-01-20,1,0,1,1,0,4,1,1,0.26,0.2727,0.56,0.0000
2,2011-01-20,1,0,1,2,0,4,1,1,0.26,0.2727,0.56,0.0000
3,2011-01-20,1,0,1,3,0,4,1,1,0.26,0.2576,0.56,0.1642
4,2011-01-20,1,0,1,4,0,4,1,1,0.26,0.2576,0.56,0.1642


In [ ]:
df_eval = df_eval.drop(columns=["dteday", "yr", "mnth"])

# Data without the target column
data_inputs = torch.from_numpy(df_eval.values)
data_inputs = data_inputs.to(device)

# Model predictions
with torch.no_grad():
    preds = model(data_inputs.float()).squeeze(dim=1)

print(preds)

# Save predictions to a CSV file
pd.DataFrame(preds.cpu()).to_csv("sroda_cisek_wiecyk_preds.csv", index=False, header=None)

tensor([17.4314,  8.6240,  5.3688,  ..., 70.5292, 48.4567, 31.7086],
       device='cuda:0')
